# Weighted Survey Analysis

We are going to analyze American Community Survey microdata from 2017

You can learn more about that data here: https://www.census.gov/programs-surveys/acs/data/pums.html

This data is survey weighted, meaning that not every person should count equally. 

We will: 
* Examine a couple ways to correctly analyze survey data. 
* Use this data to find the average and median age of Americans in 2017.
* Find the confidence we can have in that estimate, given that it is based on a survey and not the whole population

## Load libraries needed for analysis

Tidverse is generally useful for data manipulation

srvyr is a package for weighted survey analysis

There is a good vignette on how to use it here https://github.com/gergness/srvyr


In [3]:
packages <- c("tidyverse", "srvyr")
new_packages <- packages[!(packages %in% installed.packages()[,"Package"])]
if(length(new_packages)) install.packages(new_packages)

# the semicolon on the end of this line supresses warning messages
invisible(lapply(packages, library, character.only = TRUE));

Installing package into ‘/usr/local/lib/R/3.6/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘survey’


Attaching package: ‘srvyr’

The following object is masked from ‘package:stats’:

    filter



### Read in the survey data

In [2]:
load("ACS_AgeData_2017.Rdata")

Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'ACS_AgeData_2017.Rdata', probable reason 'No such file or directory'”

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


## Check how the data looks

It has about 3.2 million rows, which is a 1% sample of the US

* `perwt` is the variable that the survey data need to be weighted by to make it accurate
* `age` is the age of every person

In [8]:
head(df)
str(df)

age,perwt
<dbl>,<int>
73,206
31,45
41,136
48,121
16,111
37,18


Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3190040 obs. of  2 variables:
 $ age  : num  73 31 41 48 16 37 18 17 7 3 ...
 $ perwt: int  206 45 136 121 111 18 13 23 21 27 ...


## Calculate the average age of the population

To calculate the average age of the population is simple!

We need to sum `perwt` and then divide that by `perwt` times the `age` for each row

This gives those people with a higher `perwt` more weight

In [1]:

df_age <- df %>% 
    # summarize allows you get summary statistics about the dataset. In this case, we calculate the weighted total number of people represented in the survey by summing perwt, and get the weighted sum of the total ages of all people 
    summarize(count = sum(perwt),
             agetotal = sum(perwt*age)) %>%
   
    # divide the weighted total of ages by the weighted total of people to get the average number of people
    mutate(averageage = agetotal/count)

# Look at the resulting number of people in the US
df_age$count

# Look at the resulting number for average age
df_age$averageage


ERROR: Error in df %>% summarize(count = sum(perwt), agetotal = sum(perwt * age)) %>% : could not find function "%>%"


You get the estimate quickly in this way, but you don't get a confidence interval

It is also not possible to easily estimate the median

## Getting more complicated statistics

The easiest way to get more complicated statistics besides the average is to use a package. There are several survey packages but my favorite is "srvyr"

The first thing we need to is take the data and turn it into a survey object


In [9]:
df_survey <- df %>% as_survey(weights = perwt)

### Calculating percentiles
We can now easily calculate more complicated statistics like the median or the 10th and 90th percentile

In [10]:
df_survey %>%
    # Calculate 10th and 90th percentiles and the median. The result automatically gives the standard error of the estimate 
    summarise(age_10th = survey_quantile(age, .1),
              age_median = survey_median(age),
              age_90th = survey_quantile(age, .9))


age_10th_q10,age_10th_q10_se,age_median,age_median_se,age_90th_q90,age_90th_q90_se
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,0,38,0,70,0


### Calculating a confidence interval

If you want the confidence interval for your estimate just add `vartype = c("ci")` to the survey_quantile arguement.

In this case it doesn't really make a difference, the 90th percentile estimate is the same as the 10th. It's a big dataset, so it's very accurate.


In [13]:
df_survey %>%
    summarise(
        age_10th = survey_quantile(age, .1, vartype = c("ci"))
    )

age_10th_q10,age_10th_q10_low,age_10th_q10_upp
<dbl>,<dbl>,<dbl>
8,8,8
